# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,929.040566,0.880034,1234,929.920600,928.160532,1859.841200,1856.321063
6,479.645374,-0.089024,1234,479.556351,479.734398,959.112701,959.468797
7,970.144271,-0.327532,1234,969.816739,970.471803,1939.633477,1940.943607
8,256.188678,0.016523,1234,256.205202,256.172155,512.410403,512.344310
9,550.589295,0.613673,1234,551.202968,549.975621,1102.405936,1099.951242
10,389.544413,-0.879341,1234,388.665072,390.423753,777.330144,780.847506
11,223.184164,-0.928682,1234,222.255482,224.112846,444.510964,448.225692
12,720.821732,0.443732,1234,721.265464,720.378000,1442.530928,1440.756001
13,59.110168,-0.549817,1234,58.560351,59.659985,117.120702,119.319971
14,277.982260,0.902977,1234,278.885236,277.079283,557.770473,554.158566


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-929.920600,928.160532
6,-479.556351,479.734398
7,-969.816739,970.471803
8,-256.205202,256.172155
9,-551.202968,549.975621
10,-388.665072,390.423753
11,-222.255482,224.112846
12,-721.265464,720.378000
13,-58.560351,59.659985
14,-278.885236,277.079283


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-929.920600,928.160532
6,-479.556351,479.734398
7,-969.816739,970.471803
8,-256.205202,256.172155
9,-551.202968,549.975621
10,-388.665072,390.423753
11,-222.255482,224.112846
12,-721.265464,720.378000
13,-58.560351,59.659985
14,-278.885236,277.079283


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-929.92060007,  928.16053157],
       [-479.55635059,  479.73439839],
       [-969.8167386 ,  970.47180328],
       [-256.20520159,  256.17215492],
       [-551.20296805,  549.97562108],
       [-388.66507185,  390.42375316],
       [-222.25548182,  224.11284614],
       [-721.26546405,  720.37800046],
       [ -58.5603508 ,   59.65998537],
       [-278.88523629,  277.07928306]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1857.841200,-1856.321063
6,957.112701,-959.468797
7,1937.633477,-1940.943607
8,510.410403,-512.344310
9,1100.405936,-1099.951242
10,775.330144,-780.847506
11,442.510964,-448.225692
12,1440.530928,-1440.756001
13,115.120702,-119.319971
14,555.770473,-554.158566


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.929041,0.940017
6,0.479645,0.455488
7,0.970144,0.336234
8,0.256189,0.508262
9,0.550589,0.806837
10,0.389544,0.060330
11,0.223184,0.035659
12,0.720822,0.721866
13,0.059110,0.225091
14,0.277982,0.951488


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,929.040566,0.880034
6,479.645374,-0.089024
7,970.144271,-0.327532
8,256.188678,0.016523
9,550.589295,0.613673
10,389.544413,-0.879341
11,223.184164,-0.928682
12,720.821732,0.443732
13,59.110168,-0.549817
14,277.982260,0.902977


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
